# Add the necessary lines to the ini's

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import healpy as hp
import bilby as bb
import sys
sys.path.append("/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/utils")
import utils as ut

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",  # or "sans-serif" or "monospace"
    "font.serif": ["Computer Modern Roman"],
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

plt.style.use("seaborn-v0_8-paper")

In [2]:
batchPaths = ["BBH_aligned_Batch0.csv","BBH_precessing_Batch0.csv"]
alignmentArr = [1,0]

# NSBHFilePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalogs/n=1e7,NSBHs,FromSkySim50_withBilbySNRs.csv"
BBH_aligned_FilePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,aligned,sampledPlusSNR.csv"
BBH_precessing_FilePath = "/global/homes/s/seanmacb/DESC/DESC-GW/gwStreetlights/data/mockCBCCatalog_csvs/BBHs_0,precessing,sampledPlusSNR.csv"
catalogPath = "catalogList"

In [76]:
def makeLabel(batch,item,spin):
    """
    Label should follow the following structure:
    batch_item_spin
    Where each keyword means the following
    batch: the batch production of the CBC catalog, 
           where all the CBC items are children of
    item: the individual CBC within a batch, usually 
          ranging from 0-300ish
    spin: 0/1, indicating spins misaligned (0), or 
          spins aligned (1)
    """
    return "{}_{}_{}".format(batch,spin,item)

def makeOutDir(base,spec):
    return os.path.join(base,spec)

def makePriorFilePath(typ,spin):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1

    if typ=="BBH":
        if spin==1: # BBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_aligned_gwtc.prior"
        else: # BBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/bbh_precessing_gwtc.prior"
    else:
        if spin==1: # NSBH, spins aligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/nsbh_aligned_gwtc.prior"
        else: # NSBH, spins misaligned
            return "/pscratch/sd/s/seanmacb/gwCosmoDesc/lib/python3.10/site-packages/bilby/gw/prior_files/nsbh_precessing_gwtc.prior"

def selectSpin():
    return np.random.randint(0,2)
    
def makeInjection_dict(injKeys,row):
    myDict = {}
    for k in injKeys:
        # if k in ("chi_1", "chi_2"): # handling the tilt/chi issues
        #     myDict[k] = row["tilt_{}".format(k[-1])]
        if k in ["ra"]:
            myDict[k] = np.deg2rad(row["m"+k]) # SkySim is over -180 to 180 deg, while Bilby defaults to 0 to 2pi
        elif k in ["dec"]:
            myDict[k] = np.deg2rad(row["m"+k]) # Skysim is from -90 to 90 deg, while bilby is -pi/2 to pi/2
        else:
            myDict[k] = row[k]
    myDict["geocent_time"] = ut.fakeGeoCentTime()
    for ent in myDict.keys():
        myDict[ent] = float(myDict[ent])
    return myDict

def getInjection_keys(typ,spin,ppath):
    if typ not in ("BBH","NSBH"):
        raise ValueError("{} is not one of supported types: ('BBH','NSBH')".format(typ))
        return -1
    if spin not in (1,0):
        raise ValueError("{} is not one of supported spins: (0,1)".format(spin))
        return -1
    return bb.gw.prior.CBCPriorDict(ppath).sample().keys()

## Order
- makeLabel
- outDirectory
- prior file path
- injection dictionary

In [77]:
CBCType="BBH"
batchNum=0

In [78]:
toWrite = np.array([])
labels_snrs_dict = {}

for b,alignment in zip(batchPaths,alignmentArr):
    batchDF = pd.read_csv(os.path.join(os.getcwd(),catalogPath,b))
    basePath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs/batch_"+str(batchNum)
    os.makedirs(basePath,exist_ok=True)

    for item,row in batchDF.iterrows():
        fullItem = "{}_{}".format(item,CBCType)
        spin=alignment # Choose spins to be aligned (1) or misaligned (0)
        label = makeLabel(batchNum,fullItem,spin)
        outDir = makeOutDir(basePath,label)
        priorPath = makePriorFilePath(CBCType,spin)
        injKeys = getInjection_keys(CBCType,spin,priorPath)
        injDict = makeInjection_dict(injKeys,row)

        labels_snrs_dict[label] = row["Network SNR"]

        toWrite = np.append(toWrite,[label,outDir,priorPath,injDict])

# Reshape the final array
toWrite = np.reshape(toWrite,(-1,4))

In [79]:
fieldKeys = ["label","outdir","prior-file", "injection-dict"]
batch0_basepath = "/pscratch/sd/s/seanmacb/proj411ProdRuns/catalogs/batch_{}/iniFiles".format(batchNum)
if not os.path.isdir(batch0_basepath):
    os.mkdir(batch0_basepath)

In [80]:
for writeables in toWrite:
    with open("BBH_base.ini", "r") as f:
        read = f.readlines()

    writeFile = os.path.join(batch0_basepath,writeables[0])+".ini"
    
    for field,value in zip(fieldKeys,writeables):
        read.append("\n")
        read.append("{}={}\n".format(field,value))

    with open(writeFile, "w") as f:
        f.writelines(read)

In [81]:
# labels_snrs_dict
sortedDict = {k: v for k, v in sorted(labels_snrs_dict.items(), key=lambda item: item[1])}
result = [k for k in list(sortedDict.keys())[9::10]]

In [82]:
for r in result:
    print(r,sortedDict[r])

0_1_31_BBH 9.244870737472295
0_0_97_BBH 9.34503829457829
0_0_119_BBH 9.503843447746815
0_0_103_BBH 9.615898670861464
0_0_25_BBH 9.81866615966412
0_1_120_BBH 10.04282719687954
0_1_124_BBH 10.291039090633635
0_1_71_BBH 10.401757112645605
0_0_85_BBH 10.650657972317967
0_0_14_BBH 10.914989434943488
0_0_109_BBH 11.11233280294648
0_0_127_BBH 11.357499140622268
0_0_53_BBH 11.538053066279618
0_0_47_BBH 12.006943444290808
0_1_17_BBH 12.300035589483484
0_0_16_BBH 12.65319205066331
0_1_21_BBH 13.06396743629984
0_1_11_BBH 13.598488973740745
0_0_20_BBH 13.992333424038447
0_1_151_BBH 14.464497302787208
0_1_143_BBH 14.8635793243409
0_1_33_BBH 15.47656296268502
0_0_120_BBH 16.352988973760645
0_1_129_BBH 17.031176633434324
0_0_77_BBH 18.08474994096146
0_0_3_BBH 20.254961751977408
0_1_74_BBH 22.19384753253317
0_1_72_BBH 27.56834607299878
0_1_43_BBH 35.97356109495084
0_1_58_BBH 189.64833326772327


In [83]:
[print(r) for r in result]

0_1_31_BBH
0_0_97_BBH
0_0_119_BBH
0_0_103_BBH
0_0_25_BBH
0_1_120_BBH
0_1_124_BBH
0_1_71_BBH
0_0_85_BBH
0_0_14_BBH
0_0_109_BBH
0_0_127_BBH
0_0_53_BBH
0_0_47_BBH
0_1_17_BBH
0_0_16_BBH
0_1_21_BBH
0_1_11_BBH
0_0_20_BBH
0_1_151_BBH
0_1_143_BBH
0_1_33_BBH
0_0_120_BBH
0_1_129_BBH
0_0_77_BBH
0_0_3_BBH
0_1_74_BBH
0_1_72_BBH
0_1_43_BBH
0_1_58_BBH


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]